# Replicating AlexNet in Keras

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Activation, Dropout, Dense
from tensorflow.keras.layers import MaxPooling2D, Conv2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train,y_train), (x_test, y_test) = cifar10.load_data() # loading the CIFAR10 Data

print('x_train shape:', x_train.shape)    # Display the shape of the data
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10
y_train = to_categorical(y_train, num_classes) # one hot encoding
y_test = to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
l2_reg = 0
# Initialize Model
model = Sequential()

model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:], padding='same',kernel_regularizer= l2(l2_reg) ))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())     # 1st fully connected
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 96)       384       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 256)      1

In [7]:
batch_size = 64
epochs = 2
history = model.fit(x_train, y_train,
                    batch_size= batch_size,
                    epochs = epochs,
                    validation_data = (x_test, y_test),
                    shuffle = True)
model.save('cifar10_AlexNet.h5')


Epoch 1/2
782/782 [==============================] - 83s 106ms/step - loss: 0.9684 - accuracy: 0.7729 - val_loss: 1.1754 - val_accuracy: 0.6606
Epoch 2/2
782/782 [==============================] - 85s 109ms/step - loss: 0.9617 - accuracy: 0.7773 - val_loss: 1.1718 - val_accuracy: 0.6607


In [8]:
scores = model.evaluate(x_test, y_test, verbose = 1)
print('Test Loss: ', scores[0])
print('Test Accuracy: ', scores[1])

313/313 [==============================] - 6s 21ms/step - loss: 1.1718 - accuracy: 0.6606
Test Loss:  1.1717729568481445
Test Accuracy:  0.6606000065803528
